# Testis Pilot scRNA-Seq

## Testis Using Cell Ranger Cell Calls
* [Replicate 1 Summary](http://geneticsunderground.com/larval_gonad/01.01-testis1_scrnaseq_analysis.html)
* [Replicate 2 Summary](http://geneticsunderground.com/larval_gonad/01.02-testis2_scrnaseq_analysis.html)
* [Replicate 3 Summary](http://geneticsunderground.com/larval_gonad/01.03-testis3_scrnaseq_analysis.html)

## Testis Using Forced Cell Calls